# Data Science Challenge

**Objetivo: Desarrollar un script en Python que utilice aprendizaje automático para predecir si un cliente presentará un reclamo de seguro en el próximo año.**

Descripción: La compañía aseguradora está interesada en predecir si un cliente presentará un reclamo de seguro en el próximo año para poder tomar medidas preventivas y mejorar la satisfacción del cliente. La compañía ha recopilado datos históricos de clientes y reclamos y quiere utilizarlos para desarrollar un modelo de aprendizaje automático que pueda predecir si un cliente presentará un reclamo de seguro en el próximo año.

## Instalar las dependencias

In [1]:
!pip install -r requirements.txt

     ---------------------------------------- 7.2/7.2 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 10.2/10.2 MB 7.8 MB/s eta 0:00:00
     ------------------------------------- 324.4/324.4 kB 19.6 MB/s eta 0:00:00
     --------------------------------------- 31.4/31.4 MB 11.1 MB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 35.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [632 lines of output]
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
       ---------------------------------------- 1.1/1.1 MB 17.1 MB/s eta 0:00:00
       

In [2]:
!pip install pycaret

     -------------------------------------- 320.2/320.2 kB 6.7 MB/s eta 0:00:00
     ------------------------------------- 213.4/213.4 kB 12.7 MB/s eta 0:00:00
     ------------------------------------- 636.8/636.8 kB 13.3 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 20.7 MB/s eta 0:00:00
     ------------------------------------- 282.6/282.6 kB 17.0 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 21.6 MB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 24.4 MB/s eta 0:00:00
  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [622 lines of output]
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
  Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
    Using cached setuptools-67.5.1-py3-none-any.whl (1.1 MB)
    Using cached wheel-0.38

# Entendimiento inicial de los datos

## Cargar Librerias

In [ ]:
# %matplotlib inline

import re
import random
from collections import Counter

import numpy as np
import pandas as pd

from scipy.stats import kstest

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

import warnings
warnings.filterwarnings("ignore")

#from pycaret.classification import *
#import shap

## Definición de funciones

## Lectura y validación de datos

In [19]:
# Ruta donde se encuentra alojado el dataset del caso
url_data='../data/'

In [16]:
# Lectura y carga de la data
df=pd.read_csv(url_data+'datos_de_prueba.csv', na_values='?', low_memory=False)

In [17]:
# Validar la correcta lectura del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   ID del cliente                3000 non-null   int64 
 1   Sexo                          3000 non-null   object
 2   Edad                          3000 non-null   int64 
 3   Ingresos                      3000 non-null   int64 
 4   Tipo de hogar                 3000 non-null   object
 5   Estado civil                  3000 non-null   object
 6   Número de hijos               3000 non-null   int64 
 7   Puntuación de crédito         3000 non-null   int64 
 8   Tipo de trabajo               3000 non-null   object
 9   Educación                     3000 non-null   object
 10  Fecha de inicio de la póliza  3000 non-null   object
 11  Tipo de seguro                3000 non-null   object
 12  Monto del seguro              3000 non-null   int64 
 13  Fecha del reclamo 

Se observa que se tienen 3000 filas y 15 columnas de los tipos int64(7), object(8). Se tiene que modificar el tipo de dato de las variables: ID de cliente, Fecha de inicio de la póliza, Fecha del reclamo para su posterior tratamiento.

No se tienen valores vacios.

In [18]:
# Mostrar los 5 primeros registros
df.head(5)

,ID del cliente,Sexo,Edad,Ingresos,Tipo de hogar,Estado civil,Número de hijos,Puntuación de crédito,Tipo de trabajo,Educación,Fecha de inicio de la póliza,Tipo de seguro,Monto del seguro,Fecha del reclamo,Monto del reclamo
0,1,Femenino,28,558900,Departamento,Viudo,3,556,Empresario,Preparatoria,2022-03-28,Vida,1806,2022-11-14,558
1,2,Femenino,18,212925,Departamento,Viudo,1,330,Empleado,Posgrado,2022-09-15,Vida,1371,2022-04-25,922
2,3,Femenino,37,838328,Departamento,Casado,3,776,Empresario,Universidad,2022-03-18,Hogar,6751,2022-07-31,3128
3,4,Masculino,25,459616,Casa,Soltero,4,821,Independiente,Universidad,2022-11-14,Hogar,5448,2022-07-29,1493
4,5,Masculino,41,678250,Casa,Casado,4,580,Empresario,Universidad,2023-03-02,Vida,5625,2022-07-08,1907


## Estudiar las dimensiones del dataset

In [20]:
df.shape

(3000, 15)

Se tienen 3000 filas y 15 columnas

In [21]:
df.size

45000

In [24]:
df['ID del cliente'].value_counts()

1       1
2004    1
1995    1
1996    1
1997    1
       ..
1003    1
1004    1
1005    1
1006    1
3000    1
Name: ID del cliente, Length: 3000, dtype: int64

El ID correspondiente al análisis es el ID del cliente, se aprecia que cada ID es único por lo que **cada fila corresponde a un único cliente o reclamo histórico.**

## Evaluar el tipo de problema

In [28]:
df.head(3)

,ID del cliente,Sexo,Edad,Ingresos,Tipo de hogar,Estado civil,Número de hijos,Puntuación de crédito,Tipo de trabajo,Educación,Fecha de inicio de la póliza,Tipo de seguro,Monto del seguro,Fecha del reclamo,Monto del reclamo
0,1,Femenino,28,558900,Departamento,Viudo,3,556,Empresario,Preparatoria,2022-03-28,Vida,1806,2022-11-14,558
1,2,Femenino,18,212925,Departamento,Viudo,1,330,Empleado,Posgrado,2022-09-15,Vida,1371,2022-04-25,922
2,3,Femenino,37,838328,Departamento,Casado,3,776,Empresario,Universidad,2022-03-18,Hogar,6751,2022-07-31,3128


## Variable objetivo

In [35]:
df['Fecha de inicio de la póliza']=pd.to_datetime(df['Fecha de inicio de la póliza'])
df['Fecha del reclamo']=pd.to_datetime(df['Fecha del reclamo'])

In [37]:
df.head(20)

,ID del cliente,Sexo,Edad,Ingresos,Tipo de hogar,Estado civil,Número de hijos,Puntuación de crédito,Tipo de trabajo,Educación,Fecha de inicio de la póliza,Tipo de seguro,Monto del seguro,Fecha del reclamo,Monto del reclamo
0,1,Femenino,28,558900,Departamento,Viudo,3,556,Empresario,Preparatoria,2022-03-28,Vida,1806,2022-11-14,558
1,2,Femenino,18,212925,Departamento,Viudo,1,330,Empleado,Posgrado,2022-09-15,Vida,1371,2022-04-25,922
2,3,Femenino,37,838328,Departamento,Casado,3,776,Empresario,Universidad,2022-03-18,Hogar,6751,2022-07-31,3128
3,4,Masculino,25,459616,Casa,Soltero,4,821,Independiente,Universidad,2022-11-14,Hogar,5448,2022-07-29,1493
4,5,Masculino,41,678250,Casa,Casado,4,580,Empresario,Universidad,2023-03-02,Vida,5625,2022-07-08,1907
5,6,Masculino,58,322007,Departamento,Soltero,3,445,Empleado,Posgrado,2022-09-29,Auto,6316,2022-09-06,4049
6,7,Masculino,39,813734,Departamento,Casado,0,663,Independiente,Posgrado,2022-03-28,Vida,2354,2022-04-05,1153
7,8,Femenino,21,238991,Departamento,Viudo,1,759,Independiente,Preparatoria,2022-06-08,Hogar,9248,2022-12-12,6341
8,9,Femenino,27,523721,Departamento,Soltero,1,569,Empleado,Universidad,2022-12-24,Salud,2713,2022-12-19,327
9,10,Masculino,44,690177,Casa,Divorciado,0,522,Independiente,Posgrado,2022-08-31,Vida,9431,2022-06-15,1918
